In [1]:
#!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install numpy --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg/numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl

!pip install pylibjpeg-libjpeg==2.2.0 --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg
!pip install pylibjpeg-openjpeg==2.3.0 --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg

!pip install python-gdcm --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/pydicom_working_combo/kaggle/working/mysitepackages/python_gdcm
!pip install pylibjpeg --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/pydicom_working_combo/kaggle/working/mysitepackages/pylibjpeg

!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
!pip install pydicom --no-index --find-links=/kaggle/input/suman-dicom-tpu-supporting-combo/pydicom_working_combo/kaggle/working/mysitepackages/pydicom

!pip install tensorflow-io==0.37.1 --no-index --find-links=/kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow_io

!pip install tensorflow==2.16.1 --no-index --find-links=/kaggle/input/suman-tensorflow-working-combo/kaggle/working/mysitepackages/tensorflow

Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg/numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg
Processing /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_libjpeg/pylibjpeg_libjpeg-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg
Processing /kaggle/input/suman-dicom-tpu-supporting-combo/dicom_tpu_supporting_combo/kaggle/working/mysitepackages/pylibjpeg_openjpeg/pylibjpeg_openjpeg-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Looking in links: /kaggle/input/suman-dicom-tpu-supporting-combo/pydicom_

In [ ]:
"""!pip download python-gdcm --no-deps -d /kaggle/working/mysitepackages/python_gdcm
!pip download pylibjpeg --no-deps -d /kaggle/working/mysitepackages/pylibjpeg
!pip download pydicom --no-deps -d /kaggle/working/mysitepackages/pydicom"""

In [ ]:
"""from zipfile import ZipFile

dirName = '/kaggle/working/mysitepackages'
zipName = 'pydicom_working_combo.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)"""

In [2]:
import pydicom
from pydicom import dcmread
from pydicom.dataset import FileDataset, FileMetaDataset
from pydicom.uid import generate_uid, ImplicitVRLittleEndian

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor
import datetime
from typing import Tuple, Optional
import warnings
warnings.filterwarnings('ignore')
import shutil

#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
#from sklearn.metrics import log_loss
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

In [3]:
print(tf.__version__)
print(tfio.__version__)

2.16.1
0.37.1


In [6]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers

WEIGHTS_PATH = "/kaggle/input/suman-keras-inception-net-v3-weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
bias_list = [-9.21101, -7.87573, -7.83279, -9.43417, -9.83967, -8.16937, -9.43417
             , 6.085, -9.24492, -8.20532, -9.83967, -9.24492, -9.14646, -8.20532]

rsna_input = layers.Input(shape=(128,128,3), name="rsna_input")
conv_base = InceptionV3(include_top=False, weights=WEIGHTS_PATH, input_tensor=rsna_input, input_shape=(128,128,3))
conv_base.trainable = False

gap = layers.GlobalAveragePooling2D()(conv_base.output)

flattened_layer = layers.Flatten()(gap)
hidden_layer3 = layers.Dense(64, activation="selu", kernel_initializer=keras.initializers.LecunNormal(seed=None))(flattened_layer)
hidden_layer3 = layers.BatchNormalization()(hidden_layer3)
rsna_output = layers.Dense(14, activation="softmax", bias_initializer=tf.keras.initializers.Constant(np.array(bias_list)))(hidden_layer3)
model = tf.keras.Model(rsna_input, rsna_output)

model.load_weights('/kaggle/input/rsna-aneurysm-cta-base/tensorflow2/inception-v3-transfer-learning/3/keras_inceptionnet_baseline_v0_2.keras')

In [ ]:
"""def custom_auc_metric():
        return tf.keras.metrics.AUC(multi_label=True, num_labels=14, from_logits=False
                                    , label_weights=[1,1,1,1,1,1,1,13,1,1,1,1,1,1])

model = keras.models.\
load_model("/kaggle/input/rsna-aneurysm-cta-base/tensorflow2/inception-v3-transfer-learning/2/keras_inceptionnet_baseline.keras"
           , custom_objects={'AUC': custom_auc_metric})"""

In [7]:
from multiprocessing import cpu_count
n_cores = cpu_count()
print(f'Number of Logical CPU cores: {n_cores}')

Number of Logical CPU cores: 4


In [8]:
allowed_tags = ['BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID', 'HighBit', 'ImageOrientationPatient'
                , 'ImagePositionPatient', 'InstanceNumber', 'Modality', 'PhotometricInterpretation'
                , 'PixelRepresentation', 'PixelSpacing', 'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope'
                , 'RescaleType', 'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices']

In [9]:
pos_dict = {0: 'anterior_communicating_artery',
 1: 'basilar_tip',
 2: 'left_anterior_cerebral_artery',
 3: 'left_infraclinoid_internal_carotid_artery',
 4: 'left_middle_cerebral_artery',
 5: 'left_posterior_communicating_artery',
 6: 'left_supraclinoid_internal_carotid_artery',
 7: 'aneurysm_present',
 8: 'other_posterior_circulation',
 9: 'right_anterior_cerebral_artery',
 10: 'right_infraclinoid_internal_carotid_artery',
 11: 'right_middle_cerebral_artery',
 12: 'right_posterior_communicating_artery',
 13: 'right_supraclinoid_internal_carotid_artery'}

SUBMISSION_COLS = [
    'SeriesInstanceUID',
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present',
]

LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present',
]

In [10]:
BATCH_SIZE = 8

def format_column_name(snake_str):
    """Convert snake_case string to Title Case with spaces"""
    # Split the string into words and capitalize each word
    return ' '.join(word.capitalize() for word in snake_str.split('_'))

def process_mri_t2(image: np.ndarray):
    """
    Specific preprocessing for T2-weighted MRI
    - Bias field correction
    - Intensity normalization
    """
    # Convert to float32
    image = image.astype(np.float32)
    
    # Apply N4 bias field correction if needed
    # Note: This would require SimpleITK, commented out for simplicity
    # image = self._apply_bias_correction(image)
    
    # Z-score normalization
    mean = np.mean(image)
    std = np.std(image)
    image = (image - mean) / (std + 1e-10)
    
    # Scale to [0,1] range
    image = (image - np.min(image)) / (np.max(image) - np.min(image) + 1e-10)
    
    return image

def process_mri_t1_post(image: np.ndarray):
    """
    Specific preprocessing for T1 post-contrast MRI
    - Enhancement of contrast regions
    - Intensity standardization
    """
    image = image.astype(np.float32)
    
    # Enhance contrast
    p2, p98 = np.percentile(image, (2, 98))
    image = np.clip(image, p2, p98)
    
    # Normalize to [0,1]
    image = (image - p2) / (p98 - p2 + 1e-10)
    
    return image

def process_cta(image: np.ndarray, dcm: pydicom.dataset.FileDataset):
    """
    Specific preprocessing for CTA
    - Window/level optimization for vessel visualization
    - Vessel enhancement
    """
    # Get window settings (typical for CTA)
    window_center = 100  # Typical for CTA
    window_width = 700   # Typical for CTA
    
    # Override with DICOM values if available
    if hasattr(dcm, 'WindowCenter') and hasattr(dcm, 'WindowWidth'):
        window_center = dcm.WindowCenter
        window_width = dcm.WindowWidth
        
        # Handle multiple window settings
        if isinstance(window_center, pydicom.multival.MultiValue):
            window_center = window_center[0]
        if isinstance(window_width, pydicom.multival.MultiValue):
            window_width = window_width[0]
    
    # Apply window/level
    min_value = window_center - window_width // 2
    max_value = window_center + window_width // 2
    image = np.clip(image, min_value, max_value)
    
    # Normalize to [0,1]
    image = (image - min_value) / (max_value - min_value + 1e-10)
    
    return image

def process_mra(image: np.ndarray):
    """
    Specific preprocessing for MRA
    - Maximum intensity projection consideration
    - Vessel enhancement
    """
    image = image.astype(np.float32)
    
    # Normalize based on tissue characteristics
    p0, p100 = np.percentile(image, (0, 100))
    image = (image - p0) / (p100 - p0 + 1e-10)
    
    # Optional: Vessel enhancement using Frangi filter
    # Note: This would require additional dependencies
    # image = self._apply_vessel_enhancement(image)
    
    return image

def read_and_parse_dicom_files_tensorflow_train(dcm_path, f_coord=None):
    #print("dcm_path - ", dcm_path)
    """raw_image = tf.io.read_file(dcm_path)
    image = tfio.image.decode_dicom_image(raw_image, scale='auto', on_error='lossy', dtype=tf.float32)

    tag_id = tfio.image.dicom_tags.Modality
    tag_value = tfio.image.decode_dicom_data(raw_image,tag_id)
    modality = tag_value.numpy().decode('UTF-8')"""

    # Read the DICOM file
    raw_image = dcmread(dcm_path.numpy().decode('utf-8'))
    
    # Access the pixel data as a NumPy array
    image = raw_image.pixel_array
    modality = raw_image.Modality

    if modality == 'CTA':
        image = process_cta(image, dcm_path)
        
    if modality == 'MRA':
        image = process_mra(image)
        
    if modality == 'MRI T2':
        image = process_mri_t2(image)

    if modality == 'MRI T1post':
        image = process_mri_t1_post(image)

    #print(image.shape)

    if np.any(np.isnan(image)):
        print(f"Warning: NaN values found in image {dcm_path}")
        
    if np.any(np.isinf(image)):
        print(f"Warning: Infinite values found in image {dcm_path}")

    # this one is for TF DICOM read
    """specific_slice_tensor = tf.slice(
        image,
        begin=[f_coord, 0, 0, 0],
        size=[1, image.shape[1], image.shape[2], image.shape[3]] 
    )"""

    if image.ndim == 2:
        specific_slice_tensor = image
    else:
        specific_slice_tensor = tf.slice(
            image,
            begin=[f_coord, 0, 0],
            size=[1, image.shape[1], image.shape[2]]
        )
        # Squeeze to remove the extra dimension of size 1
        specific_slice_tensor = tf.squeeze(specific_slice_tensor)
    
    expanded_image = tf.expand_dims(specific_slice_tensor, -1)
    #m, M=tf.math.reduce_min(expanded_image), tf.math.reduce_max(expanded_image)
    #expanded_image = (tf.image.grayscale_to_rgb(expanded_image)-m)/(M-m)
    expanded_image = tf.image.grayscale_to_rgb(expanded_image)
    expanded_image = tf.image.resize(expanded_image, (128,128))
    #tf.print("expanded_image shape - ",expanded_image.shape)
    sqzd_image = tf.squeeze(expanded_image)
    #tf.print("sqzd_image shape - ", sqzd_image.shape)
    return sqzd_image

def load_dicom_tf(filepath_tensor, f_coord_tensor=None):
    """
    Wrapper function to call read_and_parse_dicom_files_tensorflow_train within a tf.data.Dataset pipeline.
    """
    #if f_coord_tensor:
    image = tf.py_function(
        read_and_parse_dicom_files_tensorflow_train,
        inp=[filepath_tensor, f_coord_tensor],
        Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )
    
    """else:
        image = tf.py_function(
            read_and_parse_dicom_files_tensorflow_train,
            inp=[filepath_tensor],
            Tout=tf.float32 # Adjust Tout based on the expected output type of your image data
    )"""
    # Ensure the shape is defined if known, as tf.py_function can lose shape info
    # image.set_shape([128, 128]) # Example for a 2D image, adjust as needed
    return image 

def preprocessing(dcm_path, f_coord):
    train_img = load_dicom_tf(dcm_path, f_coord)
    #train_img = read_and_parse_dicom_files_tensorflow_train(dcm_path, f_coord)
    train_img = tf.reshape(train_img, shape=(128, 128, 3))
    return train_img

def load_dataset_tensorflow_train(dcm_path, f_coord=0):
    image = preprocessing(dcm_path, f_coord)
    return {"images": tf.cast(image, tf.float32), "dcm_paths": dcm_path}

def dict_to_tuple(inputs):
    return inputs["images"], inputs["dcm_paths"]

def generate_tf_datasets(dicom_paths, batch_size = 8):
    
    image_dataset = tf.data.Dataset.from_tensor_slices((dicom_paths))
    
    image_ds = image_dataset.map(load_dataset_tensorflow_train, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
    image_ds = image_ds.batch(batch_size=BATCH_SIZE, drop_remainder=False)
    image_ds = image_ds.prefetch(tf.data.AUTOTUNE)
    
    return image_ds

def predict(series_path: str) -> pl.DataFrame | pd.DataFrame:
    print("Starting with Series -> ",series_path)
    all_filepaths = []
    for root, _, files in os.walk(series_path):
        for file in files:
            if file.endswith('.dcm'):
                all_filepaths.append(os.path.join(root, file))
    all_filepaths.sort()
    test_ds = generate_tf_datasets(all_filepaths, BATCH_SIZE)
    
    all_predictions = []
    for dicoms, dcm_paths in test_ds:
        #print(f"Images: {dicoms.numpy().shape}, DCM Paths: {dcm_paths.shape}")
        batch_predictions = model.predict(dicoms.numpy())
        for path, pred in zip(dcm_paths, batch_predictions):
            #print("path is -",path)
            pred_copy = pred.copy().reshape(1, -1)
            #pred_copy[0, 7] = 1 - pred_copy[0, 7]
            data_dict = {'DICOM Path': path.numpy().decode('utf-8').split('/')[-2]}
            data_dict.update({format_column_name(pos_dict[i]): float(pred) for i, pred in enumerate(pred_copy[0])})
            all_predictions.append(data_dict)
    
    final_df = pl.DataFrame(all_predictions)
    
    final_df.columns = SUBMISSION_COLS
    
    final_df = final_df.group_by("SeriesInstanceUID").agg(
        pl.col(LABEL_COLS[0]).max().alias(LABEL_COLS[0]),
        pl.col(LABEL_COLS[1]).max().alias(LABEL_COLS[1]),
        pl.col(LABEL_COLS[2]).max().alias(LABEL_COLS[2]),
        pl.col(LABEL_COLS[3]).max().alias(LABEL_COLS[3]),
        pl.col(LABEL_COLS[4]).max().alias(LABEL_COLS[4]),
        pl.col(LABEL_COLS[5]).max().alias(LABEL_COLS[5]),
        pl.col(LABEL_COLS[6]).max().alias(LABEL_COLS[6]),
        pl.col(LABEL_COLS[7]).max().alias(LABEL_COLS[7]),
        pl.col(LABEL_COLS[8]).max().alias(LABEL_COLS[8]),
        pl.col(LABEL_COLS[9]).max().alias(LABEL_COLS[9]),
        pl.col(LABEL_COLS[10]).max().alias(LABEL_COLS[10]),
        pl.col(LABEL_COLS[11]).max().alias(LABEL_COLS[11]),
        pl.col(LABEL_COLS[12]).max().alias(LABEL_COLS[12]),
        pl.col(LABEL_COLS[13]).max().alias(LABEL_COLS[13])
    )

    shutil.rmtree('/kaggle/shared', ignore_errors=True)

    return final_df.drop('SeriesInstanceUID')

In [ ]:
from datetime import datetime

TEST_RUN = False
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def get_all_subfolders(root_folder_path):
    """
    Retrieves a list of all subfolders (including nested ones) within a given root folder.

    Args:
        root_folder_path (str): The path to the root folder.

    Returns:
        list: A list of strings, where each string is the full path to a subfolder.
    """
    subfolders = []
    for dirpath, dirnames, _ in os.walk(root_folder_path):
        for dirname in dirnames:
            subfolders.append(os.path.join(dirpath, dirname))
    return subfolders

In [ ]:
if TEST_RUN:
    
    # Example usage:
    folder_to_scan = "/kaggle/input/rsna-intracranial-aneurysm-detection/series"  # Replace with the actual path
    #all_series_for_runtime_test = get_all_subfolders(folder_to_scan)
    print("all series list length", len(all_series_for_runtime_test))
    
    start = 110
    end = 210
    
    test_series = all_series_for_runtime_test[start:end]
    print("test run - series list length",len(test_series))
    
    start_main = datetime.now()
    
    for i in range(len(test_series)):
        start = datetime.now()
        df = predict(test_series[i])
        now = datetime.now()
        difference = (now - start).total_seconds()
        print("finished processing for series {0} in {1} sec".format(test_series[i], difference))
    
    now_main = datetime.now()
    difference_main = (now_main - start_main).total_seconds()
    print("finished processing all series in {0} sec".format(difference_main))

In [11]:
import kaggle_evaluation.rsna_inference_server
#import shutil

shutil.rmtree('/kaggle/shared', ignore_errors=True)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

inference_server = kaggle_evaluation.rsna_inference_server.RSNAInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    print("inside if")
    inference_server.serve()
else:
    print("inside else")
    inference_server.run_local_gateway()
    display(pl.read_parquet('/kaggle/working/submission.parquet'))

inside else
Starting with Series ->  /kaggle/shared/kaggle/input/rsna-intracranial-aneurysm-detection/kaggle_evaluation/series/1.2.826.0.1.3680043.8.498.10057359944432090722321480667591403108
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140

SeriesInstanceUID,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""1.2.826.0.1.3680043.8.498.1005…",0.000028,0.000103,0.000098,0.000032,0.000009,0.000439,0.000007,0.999999,0.000012,0.000122,0.000005,0.00002,0.000129,0.000016
"""1.2.826.0.1.3680043.8.498.1007…",4.5469e-7,0.000002,0.000001,1.7246e-7,3.4346e-7,2.5658e-7,2.0309e-7,0.999998,3.7202e-7,7.0125e-7,2.2901e-7,5.7795e-7,4.1412e-7,0.000011
"""1.2.826.0.1.3680043.8.498.1002…",4.8793e-7,0.000002,8.3079e-7,2.4343e-7,1.8837e-7,4.1176e-7,3.0803e-7,0.999997,4.8689e-7,5.2027e-7,1.1413e-7,8.3036e-7,5.6456e-7,0.000013
